#### SARSA

In [1]:
import gym
import numpy as np 
from keras.models import Sequential, Model
from keras.layers import Dense, Dropout, Input
from keras.layers.merge import Add, Multiply
from keras.optimizers import Adam
import keras.backend as K

import tensorflow as tf

import random
from collections import deque

Using TensorFlow backend.


In [2]:
from graphics import *
import random
import numpy as np

#### create 5x5 Grid world

In [3]:
win = GraphWin("My Window", 600,600)
win.setBackground(color_rgb(255,255,255))

In [4]:
def line(x1,y1,x2,y2):
    """
    create a line between pt(x1,y1) and pt(x2,y2) and return it
    """
    ln = Line(Point(x1,y1),Point(x2,y2))
    ln.setOutline(color_rgb(0,0,0))
    ln.setWidth(2)
    
    return ln

In [5]:
#border lines
border1 = line(50,50,550,50)
border1.draw(win)

border2 = line(550,50,550,550)
border2.draw(win)

border3 = line(50,550,550,550)
border3.draw(win)

border4 = line(50,50,50,550)
border4.draw(win)

#lines horizontal
ln1 = line(50,150,550,150)
ln1.draw(win)

ln2 = line(50,250,550,250)
ln2.draw(win)

ln3 = line(50,350,550,350)
ln3.draw(win)

ln4 = line(50,450,550,450)
ln4.draw(win)

#lines vertical

lnv1 = line(150,50,150,550)
lnv1.draw(win)

lnv2 = line(250,50,250,550)
lnv2.draw(win)

lnv3 = line(350,50,350,550)
lnv3.draw(win)

lnv4 = line(450,50,450,550)
lnv4.draw(win)

#create goal, obstacle and hole

rect_goal = Rectangle(Point(450,50), Point(550,150))
rect_goal.setFill(color_rgb(0,255,0))
rect_goal.draw(win)

rect_hole = Rectangle(Point(450,150), Point(550,250))
rect_hole.setFill(color_rgb(255,0,0))
rect_hole.draw(win)

rect_obs1 = Rectangle(Point(150,350), Point(250,450))
rect_obs1.setFill(color_rgb(0,0,0))
rect_obs1.draw(win)

rect_obs2 = Rectangle(Point(150,250), Point(250,350))
rect_obs2.setFill(color_rgb(0,0,0))
rect_obs2.draw(win)

Rectangle(Point(150.0, 250.0), Point(250.0, 350.0))

#### Create agent look like circle

In [6]:
agent = Circle(Point(100,500), 25)
agent.setFill(color_rgb(0,0,255))
agent.draw(win)

Circle(Point(100.0, 500.0), 25)

In [7]:
def up(agent):
    val = agent.getCenter()
    
    if (val.getY()) == 100 or (val.getY() == 500 and val.getX() == 200) :
        pass
    else :
        agent.move(0,-100)
        
    
def down(agent):
    val = agent.getCenter()
    
    if val.getY() == 500 or (val.getX() == 200 and val.getY() == 200):
        pass
    else :
        agent.move(0,100)

def right(agent):
    val = agent.getCenter()
    
    if val.getX() == 500 or (val.getX() == 100 and (val.getY() == 300 or val.getY() == 400)):
        pass
    else :
        agent.move(100,0)

def left(agent):
    val = agent.getCenter()
    
    if val.getX() == 100 or (val.getX() == 300 and (val.getY() == 300 or val.getY() == 400)):
        pass
    else:
        agent.move(-100,0)
        
def action(agent,val):
    if val == 0:
        up(agent)
    elif val == 1:
        down(agent)
    elif val == 2:
        right(agent)
    else :
        left(agent)

In [108]:
def step(agent,act):
    if act[0][0] > 0.1 and act[0][1] > 0.1:
        action(agent,np.random.choice([0,2]))
    elif act[0][0] > 0.1 and act[0][1] > -0.1 and act[0][1] < 0.1:
        action(agent,2)
    elif act[0][0] > 0.1 and act[0][1] < -0.1:
        action(agent,np.random.choice([1,2]))
    elif act[0][0] > -0.1 and act[0][0] < 0.1 and act[0][1] > 0.1:
        action(agent,0)
    elif act[0][0] > -0.1 and act[0][0] < 0.1 and act[0][1] > -0.1 and act[0][1] < 0.1:
        action(agent,np.random.choice([0,1]))
    elif act[0][0] > -0.1 and act[0][0] < 0.1 and act[0][1] < -0.1:
        action(agent,1)
    elif act[0][0] < -0.1 and act[0][1] > 0.1:
        action(agent, np.random.choice([0,3]))
    elif act[0][0] < -0.1 and act[0][1] > -0.1 and act[0][1] < 0.1:
        action(agent, 3)
    elif act[0][0] < -0.1 and act[0][1] < -0.1:
        action(agent, np.random.choice([1,3]))

#### Reward for agent in the each state

In [109]:
reward = np.array([[-.1,-.1,-.1,-.1,-.1],
                   [-.1,-.5,-.1,-.1,-.1],
                   [-.1,-.5,-.1,-.1,-.1],
                   [-.1,-.1,-.1,-.1,-1],
                   [-.1,-.1,-.1,-.1,1]])

In [110]:
def rewards(agent):
    val = agent.getCenter()
    i = int(val.getX() / 100) - 1
    j = int((500 - val.getY()) / 100) 
    
    
    return reward[j][i]

def observation(agent):
    val = agent.getCenter()
    i = int(val.getX() / 100) - 1
    j = int((500 - val.getY()) / 100) 
    
    return [i,j]

def env_close():
    win.close()

def reset(agent):
    agent.move(-400,400)

def set_agent(agent,obs):
    pos = agent.getCenter()
    
    pos = [pos.getX(), pos.getY()]
    
    refer_pos = [(obs[0]+1) * 100, (500 - (obs[1]*100))]
    
    
    agent.move(refer_pos[0]-pos[0], refer_pos[1] - pos[1])

#### Create the value function and policy

In [124]:
class ActorCritic:
	def __init__(self, sess):
		self.sess = sess

		self.learning_rate = 0.00006
		self.epsilon = .5
		self.epsilon_decay = 1.0
		self.gamma = .995
		self.tau   = .125

		# ===================================================================== #
		#                               Actor Model                             #
		# Chain rule: find the gradient of chaging the actor network params in  #
		# getting closest to the final value network predictions, i.e. de/dA    #
		# Calculate de/dA as = de/dC * dC/dA, where e is error, C critic, A act #
		# ===================================================================== #

		self.memory = deque(maxlen=2000)
		self.actor_state_input, self.actor_model = self.create_actor_model()
		_, self.target_actor_model = self.create_actor_model()

		self.actor_critic_grad = tf.placeholder(tf.float32, 
			[None, 2]) # where we will feed de/dC (from critic)
		
		actor_model_weights = self.actor_model.trainable_weights
		self.actor_grads = tf.gradients(self.actor_model.output, 
			actor_model_weights, -self.actor_critic_grad) # dC/dA (from actor)
		grads = zip(self.actor_grads, actor_model_weights)
		self.optimize = tf.train.AdamOptimizer(self.learning_rate).apply_gradients(grads)

		# ===================================================================== #
		#                              Critic Model                             #
		# ===================================================================== #		

		self.critic_state_input, self.critic_action_input, \
			self.critic_model = self.create_critic_model()
		_, _, self.target_critic_model = self.create_critic_model()

		self.critic_grads = tf.gradients(self.critic_model.output, 
			self.critic_action_input) # where we calcaulte de/dC for feeding above
		
		# Initialize for later gradient calculations
		self.sess.run(tf.initialize_all_variables())

	# ========================================================================= #
	#                              Model Definitions                            #
	# ========================================================================= #

	def create_actor_model(self):
		state_input = Input(shape=(2,))
		h1 = Dense(4, activation='relu') (state_input)
		h2 = Dense(12, activation='relu')(h1)
		h3 = Dense(12, activation='relu')(h2)
		h4 = Dense( 6, activation='relu')(h3)
		output = Dense(2, activation='linear')(h4)
		
		model = Model(input=state_input, output=output)
		adam  = Adam(lr=0.00003)
		model.compile(loss="mse", optimizer=adam)
		return state_input, model

	def create_critic_model(self):
		state_input = Input(shape=(2,))
		state_h1 = Dense(4,activation='relu')(state_input)
		state_h2 = Dense(12, activation='relu')(state_h1)
		
		action_input = Input(shape=(2,))
		action_h1 = Dense(4,activation='relu')(action_input)
		action_h2    = Dense(12)(action_h1)
		
		merged    = Add()([state_h1, action_h1])
		merged_h1 = Dense(24, activation='relu')(merged)
		merged_h2 = Dense(12, activation='relu')(merged_h1)
		merged_h3 = Dense(4,activation='relu')(merged_h2)
		output = Dense(1, activation='linear')(merged_h3)
		model  = Model(input=[state_input,action_input], output=output)
		
		adam  = Adam(lr=0.00003)
		model.compile(loss="mse", optimizer=adam)
		return state_input, action_input, model

	# ========================================================================= #
	#                               Model Training                              #
	# ========================================================================= #

	def remember(self, cur_state, action, reward, new_state, done):
		self.memory.append([cur_state, action, reward, new_state, done])

	def _train_actor(self, samples):
		for sample in samples:
			cur_state, action, reward, new_state, _ = sample
			predicted_action = self.actor_model.predict(cur_state)
			grads = self.sess.run(self.critic_grads, feed_dict={
				self.critic_state_input:  cur_state,
				self.critic_action_input: predicted_action
			})[0]

			self.sess.run(self.optimize, feed_dict={
				self.actor_state_input: cur_state,
				self.actor_critic_grad: grads
			})
            
	def _train_critic(self, samples):
		for sample in samples:
			cur_state, action, reward, new_state, done = sample
			if not done:
				target_action = self.target_actor_model.predict(new_state)
				future_reward = self.target_critic_model.predict(
					[new_state, target_action])[0][0]
				reward += self.gamma * future_reward
			self.critic_model.fit([cur_state, action], np.array(reward).reshape(1,1),verbose= 0)
		
	def train(self):
		batch_size = 4
		if len(self.memory) < batch_size:
			return

		rewards = []
		samples = random.sample(self.memory, batch_size)
		self._train_critic(samples)
		self._train_actor(samples)

	# ========================================================================= #
	#                         Target Model Updating                             #
	# ========================================================================= #

	def _update_actor_target(self):
		actor_model_weights  = self.actor_model.get_weights()
		actor_target_weights = self.target_actor_model.get_weights()
		
		for i in range(len(actor_target_weights)):
			actor_target_weights[i] = actor_model_weights[i]
		self.target_actor_model.set_weights(actor_target_weights)

	def _update_critic_target(self):
		critic_model_weights  = self.critic_model.get_weights()
		critic_target_weights = self.target_critic_model.get_weights()
		
		for i in range(len(critic_target_weights)):
			critic_target_weights[i] = critic_model_weights[i]
		self.target_critic_model.set_weights(critic_target_weights)		

	def update_target(self):
		self._update_actor_target()
		self._update_critic_target()

	# ========================================================================= #
	#                              Model Predictions                            #
	# ========================================================================= #

	def act(self, cur_state):
		if np.random.random() < self.epsilon:
			return np.array([np.random.uniform(-.5,.5), np.random.uniform(-.5,.5)])
		return self.actor_model.predict(cur_state)


In [125]:
sess = tf.Session()
K.set_session(sess)
actor_critic = ActorCritic(sess)

/home/gideon/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:56: UserWarning: Update your `Model` call to the Keras 2 API: `Model(inputs=Tensor("in..., outputs=Tensor("de...)`
/home/gideon/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:75: UserWarning: Update your `Model` call to the Keras 2 API: `Model(inputs=[<tf.Tenso..., outputs=Tensor("de...)`


#### Declare the variable

In [126]:
episode = 5000
max_episode_length = 50

In [ ]:
for j in range(episode):  
    
    #update the target network
    if ((j % 25) == 0):
        print(j)
        for k in range(5):
            for l in range(5):
                A = actor_critic.actor_model.predict(np.array([[k,l]]))
                val = actor_critic.critic_model.predict([np.array([[k,l]]),A])
        
                print(val ,end = "     ")
            print()
        
        actor_critic.critic_model.save_weights('./critic')
        actor_critic.actor_model.save_weights('./actor')
        actor_critic.update_target()
    
    set_agent(agent,[0,0])
    S = np.array(observation(agent))
    S = S.reshape(1,2)
    A = np.array([np.random.uniform(-5,5), np.random.uniform(-5,5)])
    A = A.reshape(1,2)
    
    for i in range(max_episode_length):  
        R = rewards(agent)
        
        A = actor_critic.act(S)
        
        A = A.reshape(1,2)
        
        step(agent,A)  
        
        S_new = np.array(observation(agent))
        S_new = S_new.reshape(1,2)
        
        
        if S_new[0][0] > -1 and S_new[0][1] > -1:
            pass
        else :
            set_agent(agent,[0,0])
            S_new[0][0] = 0
            S_new[0][0] = 0
        
        if (S[0][0] == 4) and (S[0][1] == 4):
            reset(agent)
            done = True
        else :
            done = False
        
        actor_critic.remember(S,A,R,S_new,done)
        
        actor_critic.train()
        
        if done :
            S = np.array(observation(agent))
            S = S.reshape(1,2)
            
        S = S_new
        

0
[[0.]]     [[-0.0601057]]     [[-0.12021139]]     [[-0.18031716]]     [[-0.24042279]]     
[[0.1280388]]     [[0.00373794]]     [[-0.01393016]]     [[-0.07624605]]     [[-0.1407313]]     
[[0.2560776]]     [[0.01843523]]     [[0.00747587]]     [[0.00897286]]     [[-0.02786032]]     
[[0.38411644]]     [[0.1471787]]     [[0.00971683]]     [[0.01121381]]     [[0.01271079]]     
[[0.5121552]]     [[0.26704192]]     [[0.03687047]]     [[0.01345476]]     [[0.01495175]]     
25
[[-0.01488974]]     [[0.06762932]]     [[0.15601096]]     [[0.16786297]]     [[0.13747337]]     
[[0.03866294]]     [[-0.01488974]]     [[0.01625794]]     [[0.10434854]]     [[0.19123855]]     
[[0.10298575]]     [[-0.00518486]]     [[-0.01488974]]     [[-0.01488974]]     [[0.05297877]]     
[[0.16763411]]     [[0.05916696]]     [[-0.01450955]]     [[-0.01271214]]     [[-0.01091777]]     
[[0.23403019]]     [[0.12348975]]     [[0.01528736]]     [[-0.01011688]]     [[-0.00832578]]     
50
[[-0.02178921]]     [[-0.015

In [116]:
actor_critic.memory[-1]

[array([[0, 0]]),
 array([[-0.02112696, -0.27801025]], dtype=float32),
 -0.1,
 array([[0, 0]]),
 False]

In [120]:
step(agent,np.array([[-0.0211,-0.278]]))

In [81]:
for i in range(5):
    for j in range(5):
        A = actor_critic.actor_model.predict(np.array([[i,j]]))
        val = actor_critic.critic_model.predict([np.array([[i,j]]),A])
        
        print(val,i,j ,end = "     ")
    print()

[[-48.73358]] 0 0     [[-48.35283]] 0 1     [[-48.23629]] 0 2     [[-48.18986]] 0 3     [[-48.120556]] 0 4     
[[-48.925564]] 1 0     [[-49.430687]] 1 1     [[-48.160957]] 1 2     [[-48.117188]] 1 3     [[-48.067593]] 1 4     
[[-48.55804]] 2 0     [[-49.586308]] 2 1     [[-48.988613]] 2 2     [[-48.029404]] 2 3     [[-47.98828]] 2 4     
[[-50.596508]] 3 0     [[-49.730343]] 3 1     [[-49.869175]] 3 2     [[-48.510223]] 3 3     [[-47.89785]] 3 4     
[[-53.613655]] 4 0     [[-52.427242]] 4 1     [[-51.804996]] 4 2     [[-50.085648]] 4 3     [[-48.031807]] 4 4     


#### After 300 iteration

| right| right| right| right|  GOAL  |
|------|------|------|------|--------|
| right| right|  up  |  up  |   up   |
|------|------|------|------|--------|
| left |      |  up  |  up  |  left  |
|------|------|------|------|--------|
| left |      |  up  |  up  | right  |
|------|------|------|------|--------|
| right| left |  up  | left |  left  |

#### After 1000 iteration

| right| right| right| right|  GOAL  |
|------|------|------|------|--------|
| right| right|  up  |  up  |   up   |
|------|------|------|------|--------|
|  up  |      |  up  |  up  |  left  |
|------|------|------|------|--------|
|  up  |      |  up  |  up  |  left  |
|------|------|------|------|--------|
|  up  | right|  up  | right|  right |